### Remake coordinates_bdy.nc file

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

#### Functions

In [2]:
def make_bdy_coord(t_coords, u_coords, v_coords, domain_cfg, 
                   rimwidth=1, filename='/home/users/birgal/data/NEMO/coordinates_bdy_new.nc'):
    """ Function makes a NEMO coordinate_bdy NetCDF file given input coordinates and a domain
    Arguments
    ----------
        t_coords : list
            Tuples of i,j coordinates of gridT points in the boundary. Assumes the order goes from the inside of the domain to the outside.
        u_coords : list
            Same as above but for the grid U points
        v_coords : list
            Same as above but for the grid V points
        domain_cfg : xarray dataset 
            From the domain_cfg NEMO file

    Parameters
    ----------
        filename : string 
            Specifies output file location and name
        rimwidth : integer 
            Width of your NEMO boundary (typically between 8-10 gridpoints)
            
    Returns
    ----------
        Dataset : xarray dataset
    """

    it_coords, jt_coords = np.array(list(zip(*t_coords))).astype(int)
    iu_coords, ju_coords = np.array(list(zip(*u_coords))).astype(int)
    iv_coords, jv_coords = np.array(list(zip(*v_coords))).astype(int)

    # Take subsets of the domain_cfg dataset based on the coordinates of boundary points
    domain_cfg_subset_t = domain_cfg.sel(x=xr.DataArray(it_coords, dims='z'), y=xr.DataArray(jt_coords, dims='z'))
    domain_cfg_subset_u = domain_cfg.sel(x=xr.DataArray(iu_coords, dims='z'), y=xr.DataArray(ju_coords, dims='z'))
    domain_cfg_subset_v = domain_cfg.sel(x=xr.DataArray(iv_coords, dims='z'), y=xr.DataArray(jv_coords, dims='z'))
    
    # Dimensions:
    bdy_length = int(np.floor(len(t_coords)/rimwidth))
    yb_dim     = 1 # always
    xbv_dim    = (bdy_length)  *rimwidth
    xbu_dim    = (bdy_length-1)*rimwidth
    xbt_dim    = (bdy_length)  *rimwidth
                       
    # Create xarray dataset:
    ds = xr.Dataset(
        data_vars=dict(
                    e1t=(["yb", "xbt"], domain_cfg_subset_t['e1t'].values.astype(float)),
                    e1u=(["yb", "xbu"], domain_cfg_subset_u['e1u'].values.astype(float)), 
                    e1v=(["yb", "xbv"], domain_cfg_subset_v['e1v'].values.astype(float)), 
                    e2t=(["yb", "xbt"], domain_cfg_subset_t['e2t'].values.astype(float)),
                    e2u=(["yb", "xbu"], domain_cfg_subset_u['e2u'].values.astype(float)),
                    e2v=(["yb", "xbv"], domain_cfg_subset_v['e2v'].values.astype(float)),
                    glamt=(["yb", "xbt"], domain_cfg_subset_t['glamt'].values.astype(float)),
                    glamu=(["yb", "xbu"], domain_cfg_subset_u['glamu'].values.astype(float)),
                    glamv=(["yb", "xbv"], domain_cfg_subset_v['glamv'].values.astype(float)),
                    gphit=(["yb", "xbt"], domain_cfg_subset_t['gphit'].values.astype(float)),
                    gphiu=(["yb", "xbu"], domain_cfg_subset_u['gphiu'].values.astype(float)),
                    gphiv=(["yb", "xbv"], domain_cfg_subset_v['gphiv'].values.astype(float)),                
                    nbit=(["yb", "xbt"], np.reshape(it_coords+1, (yb_dim, xbt_dim)).astype(int)),
                    nbiu=(["yb", "xbu"], np.reshape(iu_coords+1, (yb_dim, xbu_dim)).astype(int)),
                    nbiv=(["yb", "xbv"], np.reshape(iv_coords+1, (yb_dim, xbv_dim)).astype(int)),
                    nbjt=(["yb", "xbt"], np.reshape(jt_coords+1, (yb_dim, xbt_dim)).astype(int)),
                    nbju=(["yb", "xbu"], np.reshape(ju_coords+1, (yb_dim, xbu_dim)).astype(int)),
                    nbjv=(["yb", "xbv"], np.reshape(jv_coords+1, (yb_dim, xbv_dim)).astype(int)),
                    nbrt=(["yb", "xbt"], np.reshape(np.repeat(np.arange(1,rimwidth+1), bdy_length)  , (yb_dim, xbt_dim)).astype(int)),
                    nbru=(["yb", "xbu"], np.reshape(np.repeat(np.arange(1,rimwidth+1), bdy_length-1), (yb_dim, xbu_dim)).astype(int)),
                    nbrv=(["yb", "xbv"], np.reshape(np.repeat(np.arange(1,rimwidth+1), bdy_length)  , (yb_dim, xbv_dim)).astype(int)),
        ),
        attrs=dict(description="Boundary condition coordinates file"),
    )
    
    ds.to_netcdf(f'{filename}')


    return ds

### Read files
- Use coordinates from the domain_cfg.nc file or from the old boundary file

In [4]:
coord_bdy_old = xr.open_dataset('/home/users/birgal/data/NEMO/coordinates_bdy.nc')
domain_cfg    = xr.open_dataset('/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/bathymetry/domain_cfg-20231025.nc')

### Create coordinates boundary file

In [8]:
# Python coordinates of the boundary (zero-based). Assumes you go from the inside of the boundary to the outside
# T grid boundary coordinates
it_coords = (domain_cfg.x.values).astype(int)                            # indices along northern boundary
jt_coords = np.tile(domain_cfg.y.values[-1], len(it_coords)).astype(int) # boundary along constant j index at edge of domain
# U grid boundary coordinates
iu_coords = np.copy(it_coords)[:-1] # one less grid point on the east edge (I think)
ju_coords = np.copy(jt_coords)[:-1] # same as "jt" except one gridpoint shorter
# V grid boundary coordinates
iv_coords = np.copy(it_coords)   # same as "it"
jv_coords = np.copy(jt_coords)-1 # "jt - 1" for northern boundary

# Create coordinate pairs:
T_coords = list(zip(it_coords, jt_coords))
U_coords = list(zip(iu_coords, ju_coords))
V_coords = list(zip(iv_coords, jv_coords))

In [9]:
ds_coord = make_bdy_coord(T_coords, U_coords, V_coords, domain_cfg, rimwidth=1, \
                          filename='/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/old-forcing/extended/coordinates_bdy-extended.nc')

# other older stuff:

In [47]:
def make_bdy(coords, grid_bdy, grid_out, domain_cfg, bdy_type='U', time_dim=73, rimwidth=1, 
             filename='/home/users/birgal/data/boundary_conditions/bdyU_u3d_y1979_test.nc'):
    """ Function makes a NEMO boundary forcing NetCDF file for given input coordinates and saves it
    Arguments
    ----------
        coords : list
            Tuples of i,j coordinates of U or V grid points in the boundary. Assumes the order goes from the inside of the domain to the outside.
        grid_bdy : xarray dataset 
            A gridU boundary condition file to use to get the timestamps etc.

    Parameters
    ----------
        filename : string 
            Specifies output file location and name
        rimwidth : integer 
            Width of your NEMO boundary (typically between 8-10 gridpoints)
        bdy_type : string
            Specified the grid and file type for the boundary condition. Options: U, V, T, ice
            
    Returns
    ----------
        Dataset : xarray dataset
    """

    i_coords, j_coords = np.array(list(zip(*coords))).astype(int)

    # Take subsets of the domain_cfg dataset based on the coordinates of boundary points
    domain_cfg_subset = domain_cfg.sel(x=xr.DataArray(i_coords, dims='z'), y=xr.DataArray(j_coords, dims='z'))
    if bdy_type == 'U' or bdy_type == 'V' or bdy_type =='ice':
        grid_slice = grid_out.sel(time_counter=grid_out['time_counter'][0],
                                  x=xr.DataArray(i_coords, dims='z'), 
                                  y=xr.DataArray(j_coords, dims='z'))
    elif bdy_type=='T':
        grid_slice = grid_out.sel(time_counter=grid_out['time_counter'][0],
                                  x_grid_T=xr.DataArray(i_coords, dims='z'), 
                                  y_grid_T=xr.DataArray(j_coords, dims='z'))
    else:
        print('bdy_type can only be one of U, V, T, or ice')
        return
        
    # Dimensions:
    bdy_length = int(np.floor(len(coords)/rimwidth))
    depth_dim  = len(domain_cfg.nav_lev)
    yb_dim     = 1  # always
    xb_dim     = bdy_length*rimwidth

    if bdy_type == 'U':
        xbdim = 'xbU'; depthdim = 'depthu'; navlon= 'nav_lon'; navlat= 'nav_lat';
    elif bdy_type =='V':
        xbdim = 'xbV'; depthdim = 'depthv'; navlon= 'nav_lon'; navlat= 'nav_lat';
    elif bdy_type =='T':
        xbdim = 'xbT'; depthdim = 'deptht'; navlon= 'nav_lon_grid_T'; navlat= 'nav_lat_grid_T';
    elif bdy_type =='ice':
        xbdim = 'xbT'; depthdim = 'deptht'; navlon= 'nav_lon'; navlat= 'nav_lat';
        
    # Create xarray dataset:
    ds = xr.Dataset(
        data_vars=dict(
                    time_counter_bnds = (["time_counter", "bnds"], grid_bdy['time_counter_bnds'].values),
                    nbidta      = (["yb", f"{xbdim}"], np.reshape(i_coords+1, (yb_dim, xb_dim)).astype(int)),
                    nbjdta      = (["yb", f"{xbdim}"], np.reshape(j_coords+1, (yb_dim, xb_dim)).astype(int)),
                    nbrdta      = (["yb", f"{xbdim}"], np.reshape(np.repeat(np.arange(1,rimwidth+1), bdy_length), 
                                                                  (yb_dim, xb_dim)).astype(int)), 
        ),
        coords=dict(
                time_counter    = grid_bdy['time_counter'].values,
                nav_lon         = (["yb", f"{xbdim}"], np.reshape(grid_slice[f'{navlon}'].values, (yb_dim, xb_dim))),
                nav_lat         = (["yb", f"{xbdim}"], np.reshape(grid_slice[f'{navlat}'].values, (yb_dim, xb_dim))),
        ),
        attrs=dict(description  = f"Grid{bdy_type} boundary condition file"),
    )

    # Add specific variables:
    if bdy_type == 'U':
        ds['depthu_bnds'] = (["depthu","bnds"], grid_bdy['depthu_bnds'].values)
        vozocrtx_fill     = np.zeros((time_dim, depth_dim, yb_dim, xb_dim));
        # vozocrtx_fill[:]  = np.reshape(grid_slice['uo2'].values, (depth_dim, yb_dim, xb_dim))
        ds['vozocrtx']    = (["time_counter", f"{depthdim}", "yb", f"{xbdim}"], vozocrtx_fill)
        ds = ds.assign_coords({"depthu" : grid_bdy['depthu'].values})
        
    elif bdy_type == 'V':
        ds['depthv_bnds'] = (["depthv","bnds"], grid_bdy['depthv_bnds'].values)
        vomecrty_fill     = np.zeros((time_dim, depth_dim, yb_dim, xb_dim));
        # vomecrty_fill[:]  = np.reshape(grid_slice['vo2'].values, (depth_dim, yb_dim, xb_dim))
        ds['vomecrty']    = (["time_counter", f"{depthdim}", "yb", f"{xbdim}"], vomecrty_fill)
        ds = ds.assign_coords({"depthv" : grid_bdy['depthv'].values})

    elif bdy_type == 'T':
        ds['deptht_bnds'] = (["deptht","bnds"], grid_bdy['deptht_bnds'].values)    
        sossh_fill        = np.empty((time_dim, yb_dim, xb_dim));
        vosaline_fill     = np.empty((time_dim, depth_dim, yb_dim, xb_dim));
        votemper_fill     = np.empty((time_dim, depth_dim, yb_dim, xb_dim));
        sossh_fill[:]     = np.reshape(grid_slice['zos'].values   , (yb_dim, xb_dim))
        vosaline_fill[:]  = np.reshape(grid_slice['so'].values    , (depth_dim, yb_dim, xb_dim))
        votemper_fill[:]  = np.reshape(grid_slice['thetao'].values, (depth_dim, yb_dim, xb_dim))
        vosaline_fill[vosaline_fill < 10] = 10
        # sossh_fill        = np.zeros((time_dim, yb_dim, xb_dim));
        # vosaline_fill     = np.ones((time_dim, depth_dim, yb_dim, xb_dim))*33;
        # votemper_fill     = np.ones((time_dim, depth_dim, yb_dim, xb_dim))*2;
        ds['sossh']    = (["time_counter", "yb", "xbT"]          , sossh_fill)
        ds['votemper'] = (["time_counter", "deptht", "yb", "xbT"], votemper_fill)
        ds['vosaline'] = (["time_counter", "deptht", "yb", "xbT"], vosaline_fill) 
        ds = ds.assign_coords({"deptht" : grid_bdy['deptht'].values})

    elif bdy_type =='ice':
        ds['siconc'] = (["time_counter", "yb", "xbT"], np.zeros((time_dim, yb_dim, xb_dim)))
        ds['sithic'] = (["time_counter", "yb", "xbT"], np.zeros((time_dim, yb_dim, xb_dim)))
        ds['sivolu'] = (["time_counter", "yb", "xbT"], np.zeros((time_dim, yb_dim, xb_dim)))
        ds['snthic'] = (["time_counter", "yb", "xbT"], np.zeros((time_dim, yb_dim, xb_dim)))
        ds['snvolu'] = (["time_counter", "yb", "xbT"], np.zeros((time_dim, yb_dim, xb_dim)))
        
    ds.to_netcdf(f'{filename}', unlimited_dims='time_counter')

    return ds

In [15]:
# Python coordinates of the boundary (zero-based). Assumes you go from the inside of the boundary to the outside
# T grid boundary coordinates
it_coords = (domain_cfg.x.values[0:1440]).astype(int)                            # indices along northern boundary
jt_coords = np.tile(domain_cfg.y.values[-3], len(it_coords)).astype(int) # boundary along constant j index at edge of domain
# U grid boundary coordinates
iu_coords = np.copy(it_coords)[:-1] # one less grid point on the east edge (I think)
ju_coords = np.copy(jt_coords)[:-1] # same as "jt" except one gridpoint shorter
# V grid boundary coordinates
iv_coords = np.copy(it_coords)   # same as "it"
jv_coords = np.copy(jt_coords)-1 # "jt - 1" for northern boundary

# Create coordinate pairs:
T_coords = list(zip(it_coords, jt_coords))
U_coords = list(zip(iu_coords, ju_coords))
V_coords = list(zip(iv_coords, jv_coords))

In [26]:
def coord_multiple_rw(coords_in, rimwidth):
    
    i_coords, j_coords = np.array(list(zip(*coords_in))).astype(int)

    i_tiled = np.tile(i_coords, rimwidth)
    j_tiled = np.repeat(np.arange(np.amax(j_coords)-10, np.amax(j_coords)+1), len(i_coords))

    coords = list(zip(i_tiled, j_tiled))
    
    return coords

In [37]:
T_coords_rw10 = coord_multiple_rw(T_coords, 10)
U_coords_rw10 = coord_multiple_rw(U_coords, 10)
V_coords_rw10 = coord_multiple_rw(V_coords, 10)

### Create ice boundary file

In [17]:
# Use the same coordinates as above
ice_out = xr.open_dataset('/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/NEMO-test3-20230914/eANT025.L121_1m_19790101_19791231_icemod.nc')
ice_bdy = xr.open_dataset('/home/users/birgal/data/forcing_NEMO/bdyT_ice_y1979.nc')

ds_ice  = make_bdy(T_coords, ice_bdy, ice_out, domain_cfg, rimwidth=1, bdy_type='ice', \
                   filename='/home/users/birgal/data/boundary_conditions/bdyT_ice_y1979_N.nc')

In [50]:
ds_ice  = make_bdy(T_coords_rw10, ice_bdy, ice_out, domain_cfg, rimwidth=10, bdy_type='ice', \
                   filename='/home/users/birgal/data/boundary_conditions/bdyT_ice_y1979_N10.nc')

### Create tracer boundary file

In [18]:
tra_out   = xr.open_dataset('/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/NEMO-test3-20230914/eANT025.L121_1m_19790101_19791231_grid_T.nc')
tra_bdy   = xr.open_dataset('/home/users/birgal/data/forcing_NEMO/bdyT_tra_y1979.nc')

ds_tra    = make_bdy(T_coords, tra_bdy, tra_out, domain_cfg, rimwidth=1, bdy_type='T',
                     filename='/home/users/birgal/data/boundary_conditions/bdyT_tra_y1979_N.nc')

In [49]:
ds_tra2   = make_bdy(T_coords_rw10, tra_bdy, tra_out, domain_cfg, rimwidth=10, bdy_type='T', \
                    filename='/home/users/birgal/data/boundary_conditions/bdyT_tra_y1979_N10.nc')

### Create U, V boundary files

In [19]:
gridU_out  = xr.open_dataset('/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/NEMO-test3-20230914/eANT025.L121_1m_19790101_19791231_grid_U.nc')
gridU_bdy  = xr.open_dataset('/home/users/birgal/data/forcing_NEMO/bdyU_u3d_y1979.nc')

ds_gridU   = make_bdy(U_coords, gridU_bdy, gridU_out, domain_cfg, bdy_type='U', rimwidth=1, time_dim=73, 
                      filename='/home/users/birgal/data/boundary_conditions/bdyU_u3d_y1979_N.nc')

In [51]:
ds_gridU2   = make_bdy(U_coords_rw10, gridU_bdy, gridU_out, domain_cfg, bdy_type='U', rimwidth=10, time_dim=73, 
                      filename='/home/users/birgal/data/boundary_conditions/bdyU_u3d_y1979_N10.nc')

In [20]:
gridV_out  = xr.open_dataset('/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/NEMO-test3-20230914/eANT025.L121_1m_19790101_19791231_grid_V.nc')
gridV_bdy  = xr.open_dataset('/home/users/birgal/data/forcing_NEMO/bdyV_u3d_y1979.nc')

ds_gridV   = make_bdy(V_coords, gridV_bdy, gridV_out, domain_cfg, bdy_type='V', rimwidth=1, time_dim=73,
                      filename='/home/users/birgal/data/boundary_conditions/bdyV_u3d_y1979_N.nc')

In [52]:
ds_gridV2   = make_bdy(V_coords_rw10, gridV_bdy, gridV_out, domain_cfg, bdy_type='V', rimwidth=10, time_dim=73,
                      filename='/home/users/birgal/data/boundary_conditions/bdyV_u3d_y1979_N10.nc')